# Scaling Analysis for GrowthRate Quantification

We have developed a growth rate analysis notebook to evaulate a single time-lapse sequences. Now we are going to scale this analysis across multiple time-lapse sequences and extract quantitative insights across multiple cell populations.

## 1. Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# install acia dependencies
%pip uninstall acia -y
%pip install acia==0.3.0

# dependencies for Omnipose segmentation
%pip install torch torchvision torchaudio # --index-url https://download.pytorch.org/whl/cpu
%pip install omnipose==1.0.6
%pip install natsort
%pip install scipy==1.11.4

# Parameters

In [ ]:
from pathlib import Path

analysis_script = str(Path("../../case_studies/01_GrowthRateQuantification/Quantify_GrowthRates.ipynb").absolute().resolve())

In [ ]:
import os
print(os.getcwd())

In [ ]:
import os
from pathlib import Path

# place to store the data
dataset_folder = Path("01_GrowthRate")

# make sure the data exists (otherwise download)
if not dataset_folder.is_dir():
    !wget -O 01_toiam_dataset.zip https://fz-juelich.sciebo.de/s/QVzBLk8n1X6g2IG/download
    !unzip 01_toiam_dataset.zip

## 1.2 Specify the analysis script

Now you have to specify the name of the analysis script you want to apply to the image data.

**Note:** If the analysis script is not located in the same folder you need to specify the path to it.

In [ ]:
analysis_script = str(Path(analysis_script).absolute().resolve())
print(analysis_script, Path(analysis_script).exists())

# 2. Information about the underlying data

We summarize the amount of underlying data

In [ ]:
image_ids = [str(p.absolute()) for p in dataset_folder.glob("*.tiff")]

## TODO: give an overview about the data
print(image_ids)

In [ ]:
#!rm -r automated_executions
#!rm -r 01_GrowthRate

# 3. Scale the analysis script to all image sequences

Now we apply the analysis script to every image sequence individually 🚀! You can lean back and enjoy the working computer 😎 🥂

**Note:** For heavy analysis scripts or for larget `datasets` this process may take a while (from minutes to hours or days). The top-level progress bar will indicate the total progress and give you an indication how long this will take. For large image data volumes we can recommend execution over night 🌔!

In [ ]:
os.environ["JYPN_NO_DEP_INSTALL"] = "True"

In [ ]:
from datetime import datetime
from pathlib import Path
from acia.analysis import scale

# set the base path for all results
stem = Path(analysis_script).stem
output_path = Path("./automated_executions")

print(f"Results are stored in: {output_path.absolute()}")

In [ ]:
# scale your analysis script to many time-lapses
result = scale(
    output_path,
    analysis_script=analysis_script,
    image_ids=image_ids,
    exist_ok=True,
    execution_naming=lambda iid: f"execution_{Path(iid).stem}",
    kernel_name="python3")

# 4. Inspect your analysis results


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

images = []

# put your analysis code for summary statistics here!
for res_path in Path("automated_executions").glob("execution_*"):
    im = Image.open(res_path / "output" / "gr_comparison.png")
    images.append(im)
     
fig, axes = plt.subplots(1, len(images), figsize=(50, 5*len(images)))

for ax, im in zip(axes, images):
    ax.imshow(im)
    
    ax.axis("off")   

In [ ]:
import pandas as pd

df_gr_estimates = []

# put your analysis code for summary statistics here!
for res_path in sorted(Path("automated_executions").glob("execution_*")):    
    df_gr_est = pd.read_csv(res_path / "output" / "growth_estimates.csv")
    df_gr_est["image_id"] = res_path.name[-4:]
    df_gr_estimates.append(df_gr_est)
    
df_gr_estimates = pd.concat(df_gr_estimates)



In [ ]:
import pandas as pd

df_all_cells = []
df_all_colonies = []

# put your analysis code for summary statistics here!
for res_path in sorted(Path("automated_executions").glob("execution_*")):    
    df_cells = pd.read_csv(res_path / "output" / "allcells.csv", sep=";")
    df_cells["image_id"] = res_path.name[-4:]
    df_all_cells.append(df_cells)



    df_colony = pd.read_csv(res_path / "output" / "colony.csv", sep=",")
    df_colony["image_id"] = res_path.name[-4:]
    df_all_colonies.append(df_colony)    
    
df_all_cells = pd.concat(df_all_cells)
df_all_colonies = pd.concat(df_all_colonies)

df_all_colonies

In [ ]:
df_cell_counts = df_all_cells.groupby(["image_id", "time"]).size().reset_index(name="cell_count")
df_cell_areas = df_all_cells.groupby(["image_id", "time"]).agg("sum").reset_index()
df_colony_areas = df_all_colonies.groupby(["image_id", "time"]).agg("sum").reset_index()

In [ ]:
import seaborn as sns
sns.lineplot(df_cell_counts, x="time", y="cell_count", hue="image_id")
plt.yscale("log")
plt.grid("True")

In [ ]:
def rename(image_id):
    
    lookup = {"3803": "00", "3804": "01", "3805": "02", "3806": "03", "3807": "04"}
    
    return lookup[image_id]

df_cell_counts["replicate"] = df_cell_counts["image_id"].apply(rename)
df_cell_areas["replicate"] = df_cell_areas["image_id"].apply(rename)
df_colony_areas["replicate"] = df_colony_areas["image_id"].apply(rename)

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(3, 7))

# gridspec inside gridspec
#fig = plt.figure()

gs0 = gridspec.GridSpec(5, 1, figure=fig)

gs00 = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs0[0:3], hspace=0.05)

ax1 = fig.add_subplot(gs00[0])
ax2 = fig.add_subplot(gs00[1], sharex=ax1)
ax3 = fig.add_subplot(gs00[2], sharex=ax1)

#gs00.tight_layout(fig)

ax4 = fig.add_subplot(gs0[3])

axes = [ax1, ax2, ax3, ax4]

#axes = [fig.add_subplot(gs[0])]
#axes += [fig.add_subplot(gs[1], sharex=axes[0]), fig.add_subplot(gs[2], sharex=axes[0]), fig.add_subplot(gs[3])]
#fig, axes = plt.subplots(4, 1, sharex=True, figsize=(3, 9))

sns.lineplot(df_cell_counts, x="time", y="cell_count", hue="replicate", ax=axes[0], linewidth=1)
axes[0].set_yscale("log")
axes[0].grid("True")

axes[0].set_ylabel("Cell Count (CC)")

sns.lineplot(df_cell_areas, x="time", y="area", hue="replicate", ax=axes[2], linewidth=1)
axes[2].set_ylabel("TSCA [$\mu m^2$]")
axes[2].set_yscale("log")
axes[2].grid("True")

sns.lineplot(df_colony_areas, x="time", y="area", hue="replicate", ax=axes[1], linewidth=1)
axes[1].set_ylabel("TCA [$\mu m^2$]")
axes[1].set_yscale("log")
axes[1].grid("True")

axes[2].set_xlabel("Time [h]")

df_gr_estimates.loc[df_gr_estimates.method=="CA", "method"] = "TCA"

sns.stripplot(df_gr_estimates, x="method", y="mu", hue="image_id", ax=axes[3])
sns.boxplot(df_gr_estimates, x="method", y="mu", ax=axes[3], color="grey")
axes[3].set_xlabel("Growth measure")
axes[3].set_ylabel("Growth rate [$h^{-1}$]")

axes[3].grid(True)


handles, labels = axes[0].get_legend_handles_labels()
#ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
#          ncol=3, fancybox=True, shadow=True)
fig.legend(handles, labels, title="Replicates", loc='lower center', ncols=3, bbox_to_anchor=(0.5, -0.0))

for ax in axes:
    ax.legend().set_visible(False)

plt.tight_layout()


plt.savefig("summary.png", dpi=600)

In [ ]:
sns.lineplot(df_colony_areas, x="time", y="area", hue="image_id")
plt.yscale("log")
plt.grid("True")

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(2, 1, figsize=(4, 8), sharex=True)

sns.stripplot(df_gr_estimates, x="image_id", y="mu", hue="method", ax=axes[0])
sns.stripplot(df_gr_estimates, x="image_id", y="R2", hue="method", ax=axes[1])
axes[0].grid(True)
axes[1].grid(True)

plt.tight_layout()

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(2, 1, figsize=(4, 8), sharex=True)

sns.stripplot(df_gr_estimates, x="method", y="mu", hue="image_id", ax=axes[0])
sns.stripplot(df_gr_estimates, x="method", y="R2", hue="image_id", ax=axes[1])
axes[0].grid(True)
axes[1].grid(True)

axes[0].set_ylabel("Growth rate $\mu$ [$h^{-1}$]")

plt.tight_layout()
plt.savefig("growth_rates.png", dpi=300)

In [ ]:
sns.stripplot(df_gr_estimates, x="image_id", y="R2", hue="method")
plt.grid(True)

## 🔁 Reproducibility Information

pip and conda environment details

In [ ]:
%pip freeze

In [ ]:
%mamba env export